In [13]:
import requests
from dotenv import load_dotenv

load_dotenv()

url = "https://yh-finance.p.rapidapi.com/auto-complete"

x_rapidapi_key = os.getenv("RAPIDAPI_KEY")

querystring = {"q":"intel","region":"US"}

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': x_rapidapi_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"explains":[],"count":15,"quotes":[{"exchange":"NMS","shortname":"Intel Corporation","quoteType":"EQUITY","symbol":"INTC","index":"quotes","score":212861.0,"typeDisp":"Equity","longname":"Intel Corporation","isYahooFinance":true},{"exchange":"PNK","shortname":"ARTIFICIAL INTELLIGENC TECH SOL","quoteType":"EQUITY","symbol":"AITX","index":"quotes","score":85902.0,"typeDisp":"Equity","longname":"Artificial Intelligence Technology Solutions Inc.","isYahooFinance":true},{"exchange":"PNK","shortname":"INTELGENX TECHNOLOGIES CORPORAT","quoteType":"EQUITY","symbol":"IGXT","index":"quotes","score":34422.0,"typeDisp":"Equity","longname":"IntelGenx Technologies Corp.","isYahooFinance":true},{"exchange":"NGM","shortname":"Intellia Therapeutics, Inc.","quoteType":"EQUITY","symbol":"NTLA","index":"quotes","score":34149.0,"typeDisp":"Equity","longname":"Intellia Therapeutics, Inc.","isYahooFinance":true},{"exchange":"NGM","shortname":"Faraday Future Intelligent Elec","quoteType":"EQUITY","symbol":"F

In [14]:
import yfinance as yf
import pandas as pd

data = yf.download("JPM BAC C WFC GS MS PNC USB COF BNY STT KEY FITB CFG TD", start="2007-12-31", end="2021-09-30")

stock_data = pd.DataFrame(data)

stock_data.drop(columns=["Volume","Open","Low","High","Close"],axis=1,level=0,inplace=True)

display(stock_data.shape)

display(stock_data)

stock_data.to_csv("stockdata.csv")

stock_data["Adj Close"].plot.line()

Exception in thread Thread-22:
Traceback (most recent call last):
Exception in thread Thread-24:
Traceback (most recent call last):
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
Exception in thread Thread-21:
Traceback (most recent call last):
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
Exception in thread Thread-20:
Traceback (most recent call last):
Exception in thread Thread-27:
Traceback (most recent call last):
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
  File "C:\Users\bmill\anaconda3\envs\dev\lib\site-packages

KeyboardInterrupt: 

In [15]:
import yahoo_fin.stock_info as si

si.get_income_statement("WFC",yearly=True)

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [11]:
import requests
import json
import pandas as pd
from sec_api import XbrlApi, QueryApi

from dotenv import load_dotenv

load_dotenv()

SEC_API_KEY = os.getenv("SEC_API_KEY")

query_api = QueryApi(api_key=SEC_API_KEY)

filing_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/0000070858/000007085821000023/bac-20201231.htm"

# get your API key at https://sec-api.io
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + SEC_API_KEY

# make request to the API
response = requests.get(final_url)

# load JSON into memory
xbrl_json = json.loads(response.text)

# income statement example
print(xbrl_json['StatementsOfIncome'])



# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        if usGaapItem == "NetIncomeLoss":
            values = []
            indicies = []
        
            
            for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
                # only consider items without segment. not required for our analysis.
                if 'segment' not in fact:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                    # ensure no index duplicates are created
                    if index not in indicies:
                        values.append(fact['value'])
                        indicies.append(index)                    

            income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 
        
    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 

income_statement = get_income_statement(xbrl_json)

# fetch all 10-Q and 10-K filings for WFC
query = {
    "query": {
        "query_string": {
            "query": "(formType:\"10-K\") AND ticker:BAC"
        }
    },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result = query_api.get_filings(query)

print(query_result)

accession_numbers = []

# extract accession numbers of each filing
for filing in query_result['filings']:
    accession_numbers.append(filing['accessionNo']);
    
# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no):
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + SEC_API_KEY

    response_tmp = requests.get(request_url)

    return json.loads(response_tmp.text)

# merge two income statements into one statement.
# row indicies of both statements have to be the same
# statement_b represents the most recent statement.
def merge_income_statements(statement_a, statement_b):
    return statement_a.merge(statement_b,
                     how="outer", 
                     #on="i_s.index", 
                     left_index=True,
                     right_index=True,
                     suffixes=('_left', ''))

# clean income statement.
# drop duplicate columns (= column name ends with "_left"), drop key_0 column, drop columns with +5 NaNs
def clean_income_statement(statement):
    for column in statement:

        # column has more than 5 NaN values
        is_nan_column = statement[column].isna().sum() > 5

        if column.endswith('_left') or column == 'key_0' or is_nan_column:
            statement = statement.drop(column, axis=1)
    
    # rearrange columns so that first column represents first quarter
    # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
    sorted_columns = sorted(statement.columns.values)
    
    return statement[sorted_columns]

# helps printing dataframes while we generate new income statement
from IPython.display import display, HTML

previous_income_statement_set = False
income_statement_final = None

for accession_no in accession_numbers:
# for accession_no in accession_numbers: # doesn't work with filings filed before 2017 - indicies not equal
    print(accession_no)
    
    # get XBRL-JSON of 10-Q or 10-K filing by accession number
    xbrl_json_data = get_xbrl_json(accession_no)
    
    # convert XBRL-JSON to a pandas dataframe
    income_statement_uncleaned = get_income_statement(xbrl_json_data)

    # clean the income statement
    income_statement_cleaned = clean_income_statement(income_statement_uncleaned)
    
    # print income statement on each iteration to monitor progress
    display(HTML(income_statement_cleaned.to_html()))
    
    # merge new income statement with previously generated income statement
    if previous_income_statement_set:
        income_statement_final = clean_income_statement(merge_income_statements(income_statement_final, income_statement_cleaned))
        income_statement_final.to_excel("file_BAC.xlsx")
    else:
        income_statement_final = income_statement_cleaned
        previous_income_statement_set = True


{'InterestAndDividendIncomeOperating': [{'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'}, 'value': '51585000000'}, {'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2019-01-01', 'endDate': '2019-12-31'}, 'value': '71236000000'}, {'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2018-01-01', 'endDate': '2018-12-31'}, 'value': '66769000000'}], 'InterestExpense': [{'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'}, 'value': '8225000000'}, {'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2019-01-01', 'endDate': '2019-12-31'}, 'value': '22345000000'}, {'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2018-01-01', 'endDate': '2018-12-31'}, 'value': '18607000000'}], 'InterestIncomeExpenseNet': [{'decimals': '-6', 'unitRef': 'usd', 'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'}, 'value': '43360000000'}, {'decimals': '-6', 'unitRef'

,2018-01-01-2018-12-31,2019-01-01-2019-12-31,2020-01-01-2020-12-31
NetIncomeLoss,28147000000,27430000000,17894000000


0000070858-20-000011


,2017-01-01-2017-12-31,2018-01-01-2018-12-31,2019-01-01-2019-12-31
NetIncomeLoss,18232000000,28147000000,27430000000


0000070858-19-000012


,2016-01-01-2016-12-31,2017-01-01-2017-12-31,2018-01-01-2018-12-31
NetIncomeLoss,17822000000,18232000000,28147000000


0000070858-18-000009


KeyError: 'StatementsOfIncome'